# Historical simulations: slope and R^2 for all months
  
Katie Brennan  
December 2020

In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
import scipy.stats as stats 
import os as os
import warnings
import matplotlib.pyplot as plt

In [ ]:
import preprocess_utils as pputils
import analysis_utils as autils 

## Load observational data (need to update with three different versions of sea ice sat data): 

In [ ]:
all_months = [1,2,3,4,5,6,7,8,9,10,11,12]
month_names = ['Jan','Feb','Mar','Apr','May','June','July','Aug','Sept','Oct','Nov','Dec','Annual']

#### t2m: 

In [ ]:
era_path = '/glade/work/rclancy/monthly_tas_ERA5.nc'
era_data = xr.load_dataset(era_path)
era_data = pputils.rename_lat_lon(era_data)
era_data = pputils.rename_dimensions(era_data, '')

In [ ]:
era_data['t2m_arc_mean'] = autils.calc_arctic_mean(era_data.t2m.isel(expver= 0),70)[:-2]
era_data['t2m_arc_mean_climo'] = era_data.t2m_arc_mean.groupby('time.month').mean()
era_data['t2m_arc_mean_anom'] = era_data.t2m_arc_mean.groupby('time.month') - era_data['t2m_arc_mean_climo']

In [ ]:
era_data['t2m_arc_mean_annual'] = era_data.t2m_arc_mean.groupby('time.year').mean()
era_data['t2m_arc_mean_annual_anom'] = era_data.t2m_arc_mean_annual - era_data.t2m_arc_mean_annual.mean(dim='year')

#### sie: 

In [ ]:
sat_path ='/glade/scratch/mkbren/cmip6_si_tas/satellite_data/Fetterer_etal_2017/sidads.colorado.edu/DATASETS/NOAA/G02135/north/monthly/data/'

for i in np.arange(1,13,1):
    sat_filename = 'N_'+f'{i:02d}'+'_extent_v3.0.csv'
    df1 = pd.read_csv(sat_path+sat_filename)
    if i ==1: 
        df2 = df1
    else: 
        df2 = df2.append(df1)

In [ ]:
df = df2.sort_values(['year',' mo'],ascending=True)
df_nonans = df.where(df[' extent']>0)
climo = df_nonans.groupby(' mo')[' extent'].mean()

df_nonans['extent_anoms'] = df_nonans[' extent'] - climo

In [ ]:
sie_1978_2020 = df_nonans[' extent'].values
sie_1978_2020_anom = np.zeros(len(sie_1978_2020))

for t in range(len(sie_1978_2020)): 
    for m in range(12):
        if t ==0: 
            m = 10
        sie_1978_2020_anom[t] = sie_1978_2020[t] - climo[m+1]

In [ ]:
era_data['sie_arc_mean_anom'] = xr.DataArray(sie_1978_2020_anom[2:-8], coords={"time":era_data.time}, dims={'time':era_data.time})
era_data['sie_arc_mean_annual'] = era_data.sie_arc_mean_anom.groupby('time.year').mean()
era_data['sie_arc_mean_annual_anom'] = era_data.sie_arc_mean_annual - era_data.sie_arc_mean_annual.mean(dim='year')

In [ ]:
era_data = era_data.isel(time=slice(None,-2))

#### Regress across full time period: 

In [ ]:
[slopes_obs, 
r_obs,
intercept_obs] = autils.scatter_linreg(era_data['t2m_arc_mean_anom'],era_data['sie_arc_mean_anom'],
                                       era_data,all_months,'obs',plotflag=False)

In [ ]:
[_, _,_] = autils.scatter_linreg(era_data['t2m_arc_mean_anom'],era_data['sie_arc_mean_anom'],era_data,[1,12],'obs',plotflag=True)

#### Regress between 1979-2014: 

In [ ]:
era_data_79_14 = era_data.sel(time=slice('1979-01-01','2014-12-31'))

[slopes_obs_79_14, 
r_obs_79_14,
intercept_obs_79_14] = autils.scatter_linreg(era_data_79_14['t2m_arc_mean_anom'],era_data_79_14['sie_arc_mean_anom'],
                                             era_data_79_14,all_months,'obs',plotflag=False)

In [ ]:
[_, _,_] = autils.scatter_linreg(era_data_79_14['t2m_arc_mean_anom'],era_data_79_14['sie_arc_mean_anom'],
                                 era_data_79_14,[1,12],'obs',plotflag=True)

In [ ]:
slope_an,intercept_annual,r_value_annual,_,_ = stats.linregress(era_data_79_14.t2m_arc_mean_annual_anom,
                                                                    era_data_79_14.sie_arc_mean_annual_anom)

slope_an, r_value_annual**2

### Load multiple satellite records: (incomplete) 

In [ ]:
# SIA all three products: 
sia_path = '/glade/work/lettier/archived_data/202003_data_paper_antarctic_sia_cmip6/processed_OBS_sia_1979-2018.nc'

#SIC NASA Team and Bootstrap monthly concentration fields here: 
path_sat_sic = '/glade/work/lettier/CDR/mon/seaice_conc_monthly_nh_1979-2018.nc'

#SIA and SIE for NASA Team and Bootstrap, Arctic and Antarctic here: 
path_sat_nt_bt = '/glade/work/lettier/CDR/mon/processed/sia_cdr_monthly_processed_nhsh_1979-2018.nc'

In [ ]:
# sat_nt_bt = xr.open_dataset(path_sat_nt_bt)

### Process CMIP6 models: 

This is creating the list of models to loop over. I've had problems with some models, and have just been removing them from the list if they cause me problems for now :) Eventually we'll need to deal with the issues and try to get them working...

In [ ]:
path = '/glade/work/mkbren/cmip6-data/cmip6-preprocessed/historical/'

preprocessed_files = os.listdir(path)
preprocessed_models = [d.split('_')[0] for d in preprocessed_files]
preprocessed_models = list(set(preprocessed_models))
# preprocessed_models.remove('ACCESS-CM2')
# preprocessed_models.remove('AWI-ESM-1-1-LR')
# preprocessed_models.remove('KIOST-ESM')
# preprocessed_models.remove('NESM3')
#preprocessed_models.remove('ACCESS-ESM1-5')
len(preprocessed_models)

In [ ]:
all_months = [1,2,3,4,5,6,7,8,9,10,11,12]
month_names = ['Jan','Feb','Mar','Apr','May','June','July','Aug','Sept','Oct','Nov','Dec','Annual']

In [ ]:
preprocessed_models = ['NorESM2-LM']

# Regress total SIE and Arctic mean temperature: 

# Full period: 
This takes a long time to run...but once its done you have all the slopes and R^2 values for all the models at your fingertips! 

### Seasonal cycle across models: 

In [ ]:
tot_sie_allmod = {}
tot_sie_allmod_ar = np.zeros((12,len(preprocessed_models)))
tot_siv_allmod = {}
tot_siv_allmod_ar = np.zeros((12,len(preprocessed_models)))

for m,mod in enumerate(preprocessed_models):
    print('Working on '+mod+'...')
    filename = mod+'_historical_siconc_sithick_tas_pan_arctic_fields.nc'
    
    data = xr.load_dataset(os.path.join(path,filename))

    tot_sie = data.sie_arc_tot*1e-6
    product = tot_sie.groupby('time.month').mean(dim='time')
    tot_sie_allmod[mod] = product
    tot_sie_allmod_ar[:,m] = product
    
    if len(data.sivol_arc_mean.shape)>1: 
        tot_siv = data.sivol_arc_mean.sel(member_id='r1i1p1f1')
    else: 
        tot_siv = data.sivol_arc_mean
    product2 = tot_siv.groupby('time.month').mean(dim='time')
    tot_siv_allmod[mod] = product2
    tot_siv_allmod_ar[:,m] = product2

#### Plot mean seasonal cycle Arctic sea ice extent for each model and satellite: 

In [ ]:
colormap = plt.cm.tab20
colors = [colormap(i) for i in np.linspace(0, 1,len(preprocessed_models))]

fig = plt.figure(figsize=(10,6))

tot_sie_MMmean = np.mean(tot_sie_allmod_ar,axis=1)

for m,mod in enumerate(preprocessed_models):
    plt.plot(np.arange(0,12,1),tot_sie_allmod[mod],label=mod, color=colors[m])

plt.plot(np.arange(0,12,1),tot_sie_MMmean,label='multi-model mean', color='k',linewidth=4)
plt.plot(np.arange(0,12,1),climo,label='Satellite (Fetterer et al 2017)', color='crimson',linewidth=4)

plt.xticks(np.arange(0, len(all_months), 1),month_names,fontsize=14,rotation='vertical')
plt.legend(bbox_to_anchor=[1.01, 1.01])
plt.ylabel('Total Arctic sea ice extent (10$^6$ km$^2$)', fontsize=16)
#plt.title('Slope: Artic mean sea ice extent vs. surface temperature', fontsize=18)

#### Plot mean seasonal cycle total Arctic sea ice volume for each model and satellite: 

In [ ]:
colormap = plt.cm.tab20
colors = [colormap(i) for i in np.linspace(0, 1,len(preprocessed_models))]

fig = plt.figure(figsize=(10,6))

tot_siv_MMmean = np.mean(tot_siv_allmod_ar,axis=1)

for m,mod in enumerate(preprocessed_models):
    plt.plot(np.arange(0,12,1),tot_siv_allmod[mod]*1e-12,label=mod, color=colors[m])

plt.plot(np.arange(0,12,1),tot_siv_MMmean*1e-12,label='multi-model mean', color='k',linewidth=4)

plt.xticks(np.arange(0, len(all_months), 1),month_names,fontsize=14,rotation='vertical')
plt.legend(bbox_to_anchor=[1.01, 1.01])
plt.ylabel('Total Arctic sea ice volume (1000 km$^3$)', fontsize=16)
#plt.title('Slope: Artic mean sea ice extent vs. surface temperature', fontsize=18)

### Put slope and R^2 values into arrays for plotting: 

## All models slope and R^2 for all months: 

In [ ]:
fig = plt.figure(figsize=(10,6))

slope_MMmean = np.mean(slopes_allmods,axis=1)
slope_MMstd = np.std(slopes_allmods,axis=1)

for i in np.arange(0,13,1):
    x = np.arange(i-0.5,i+0.59,0.1)
    y = np.ones(x.shape)
    plt.fill_between(x, y1=y*(slope_MMmean[i]+slope_MMstd[i]), 
                     y2=y*(slope_MMmean[i]-slope_MMstd[i]), 
                     alpha=0.2, color='slateblue')

for mod in preprocessed_models:
    plt.scatter(np.arange(0,13,1),slopes_plot_all[mod][1,:],label=mod, color='slateblue',marker='_', s=700)

plt.scatter(np.arange(0,13,1),slope_MMmean,label='multi-model mean', color='k',linewidth=5,marker='_', s=400)
#plt.scatter(np.arange(0,12,1),slopes_obs,label='Satellite', color='crimson',linewidth=5,marker='_', s=300)
#plt.scatter(np.arange(0,12,1),slope_MMmean_satperiod[:-1],label='multi-model mean \nsat period', color='orange',linewidth=5,marker='_', s=400)

plt.xticks(np.arange(0, len(all_months)+1, 1),month_names,fontsize=14,rotation='vertical')
plt.legend(bbox_to_anchor=[1.01, 1.01])
plt.ylabel('Slope (10$^6$ km$^2$ per $^{o}$C)', fontsize=16)
plt.title('Slope: Historical full', fontsize=18)
plt.xlim(-0.75,12.75)
plt.ylim(-5.5,0.25)

#    plt.yticks([0.5,1.5],['R$^2$','Slope'],fontsize=14)

In [ ]:
fig = plt.figure(figsize=(10,6))

r2_MMmean = np.mean(r2_allmods,axis=1)
r2_MMstd = np.std(r2_allmods,axis=1)

for i in np.arange(0,13,1):
    x = np.arange(i-0.5,i+0.59,0.1)
    y = np.ones(x.shape)
    plt.fill_between(x, y1=y*(r2_MMmean[i]+r2_MMstd[i]), 
                     y2=y*(r2_MMmean[i]-r2_MMstd[i]), 
                     alpha=0.2, color='dodgerblue')

for mod in preprocessed_models:
    plt.scatter(np.arange(0,13,1),slopes_plot_all[mod][0,:],label=mod, color='dodgerblue',marker='_', s=700)

plt.scatter(np.arange(0,13,1),r2_MMmean,label='multi-model mean', color='k',linewidth=5,marker='_', s=400)
#plt.scatter(np.arange(0,12,1),np.power(r_obs,2),label='Satellite', color='crimson',linewidth=5,marker='_', s=300)
#plt.scatter(np.arange(0,12,1),r2_MMmean_satperiod[:-1],label='multi-model mean \nsat period', color='orange',linewidth=5,marker='_', s=400)

#plt.axhline(0.5, color='k',linestyle='--')
plt.xticks(np.arange(0, len(all_months)+1, 1),month_names,fontsize=14,rotation='vertical')
plt.legend(bbox_to_anchor=[1.01, 1.01])
plt.ylabel('R$^2$ value', fontsize=16)
plt.title('R$^2$ value: Historical full', fontsize=18)
plt.xlim(-0.75,12.75)
plt.ylim(-0.05,1)

#    plt.yticks([0.5,1.5],['R$^2$','Slope'],fontsize=14)

## Investigating model repeats: 

In [ ]:
preprocessed_models_select = ['MIROC6','CESM2-WACCM','EC-Earth3-Veg-LR','CMCC-CM2-SR5','MPI-ESM-1-2-HAM','NorCPM1','CESM2',
                              'CESM2-FV2','EC-Earth3-Veg','MPI-ESM1-2-HR','CIESM','EC-Earth3','E3SM-1-0','EC-Earth3-AerChem',
                              'MRI-ESM2-0','CESM2-WACCM-FV2','MPI-ESM1-2-LR','NorESM2-LM','ACCESS-ESM1-5','E3SM-1-1-ECA','E3SM-1-1',
                              'NorESM2-MM']

In [ ]:
preprocessed_models_select = ['EC-Earth3-Veg-LR','EC-Earth3-Veg','EC-Earth3','EC-Earth3-AerChem']
# preprocessed_models_select = ['CESM2-WACCM','CESM2','CESM2-FV2','CESM2-WACCM-FV2']
# preprocessed_models_select = ['NorCPM1','NorESM2-LM','NorESM2-MM']
preprocessed_models_select = ['MPI-ESM-1-2-HAM','MPI-ESM1-2-HR','MPI-ESM1-2-LR']

fig = plt.figure(figsize=(10,6))

slope_MMmean = np.mean(slopes_allmods,axis=1)

for mod in preprocessed_models_select:
    plt.plot(slopes_plot_all[mod][1,:],label=mod)

plt.xticks(np.arange(0, len(all_months), 1) + 0.5,month_names,fontsize=14,rotation='vertical')
plt.legend(bbox_to_anchor=[1.0, 1.0], fontsize=14)
plt.ylabel('Slope (10$^6$ km$^2$ per $^{o}$C)', fontsize=14)
#    plt.yticks([0.5,1.5],['R$^2$','Slope'],fontsize=14)

In [ ]:
fig = plt.figure(figsize=(10,6))

r2_MMmean = np.mean(r2_allmods,axis=1)

for mod in preprocessed_models_select:
    plt.plot(slopes_plot_all[mod][0,:],label=mod)

#plt.plot(r2_MMmean,label='multi-model mean', color='k',linewidth=3)

plt.axhline(0.5, color='k',linestyle='--')
plt.xticks(np.arange(0, len(all_months), 1) + 0.5,month_names,fontsize=14,rotation='vertical')
plt.legend(bbox_to_anchor=[1.0, 1.0],fontsize=14)
plt.ylabel('R$^2$ value', fontsize=14)
#    plt.yticks([0.5,1.5],['R$^2$','Slope'],fontsize=14)

## Heat maps for each model: 

In [ ]:
for i,mod in enumerate(preprocessed_models):
    # Plot slopes for all models
    slopes_plot = np.zeros((2,12))

    for im, m in enumerate(all_months):
        slopes_plot[1,im] = slopes_all[mod][im]
        slopes_plot[0,im] = r_all[mod][im]**2
        
    fig = plt.figure(figsize=(10,2))
    plt.pcolormesh(slopes_plot, cmap='RdBu')
    plt.clim(1.0, -1.0)
    plt.xticks(np.arange(0, len(all_months), 1) + 0.5,month_names,fontsize=14, 
               rotation='vertical')
    plt.yticks([0.5,1.5],['R$^2$','Slope'],fontsize=14)
    plt.title(mod, fontsize=18)
    plt.colorbar(label='Slope ((10$^{6}$ km$^{2}$)/K)')

# Satellite period: 

In [ ]:
slopes_all_satperiod = {}
r_all_satperiod = {}
intercept_all_satperiod = {}

slope_annual_satperiod = {}
r_annual_satperiod = {}
intercept_annual_satperiod = {}

for mod in preprocessed_models:
    print('Working on '+mod+'...')
    filename = mod+'_historical_siconc_sithick_tas_pan_arctic_fields.nc'
    
    data = xr.load_dataset(os.path.join(path,filename))
    data = data.sel(time = slice("1979-01-01", "2018-12-01"))

    [slopes_all_satperiod[mod], 
    r_all_satperiod[mod],
    intercept_all_satperiod[mod]] = autils.scatter_linreg(data.tas_arc_mean_anom,data.sie_arc_tot*1e-6,data,
                                                          all_months,mod,plotflag=False)
    
    tas_annual_mean = data.tas_arc_mean.groupby('time.year').mean()
    tas_annual_mean_anom = tas_annual_mean - tas_annual_mean.mean(dim='year')
    sie_annual_mean = data.sie_arc_tot .groupby('time.year').mean()
    sie_annual_mean_anom = sie_annual_mean - sie_annual_mean.mean(dim='year')
    
    [slope_annual_satperiod[mod],
    intercept_annual_satperiod[mod],
    r_annual_satperiod[mod],_,_] = stats.linregress(tas_annual_mean_anom,sie_annual_mean_anom*1e-6)

In [ ]:
slopes_plot_all_satperiod = {}
slopes_allmods_satperiod = np.zeros((13,len(preprocessed_models)))
r2_allmods_satperiod = np.zeros((13,len(preprocessed_models)))

for i,mod in enumerate(preprocessed_models):
    # Plot slopes for all models
    slopes_plot = np.zeros((2,13))

    for im, m in enumerate(all_months):
        slopes_plot[1,im] = slopes_all_satperiod[mod][im]
        slopes_plot[0,im] = r_all_satperiod[mod][im]**2
    
    slopes_plot[1,12] = slope_annual_satperiod[mod]
    slopes_plot[0,12] = r_annual_satperiod[mod]**2
        
    slopes_plot_all_satperiod[mod] = slopes_plot
    slopes_allmods_satperiod[:,i] = slopes_plot[1,:]
    r2_allmods_satperiod[:,i] = slopes_plot[0,:]

In [ ]:
fig = plt.figure(figsize=(10,6))

slope_MMmean_satperiod = np.mean(slopes_allmods_satperiod,axis=1)
slope_MMstd_satperiod = np.std(slopes_allmods_satperiod,axis=1)

for i in np.arange(0,13,1):
    x = np.arange(i-0.5,i+0.59,0.1)
    y = np.ones(x.shape)
    plt.fill_between(x, y1=y*(slope_MMmean_satperiod[i]+slope_MMstd_satperiod[i]), 
                     y2=y*(slope_MMmean_satperiod[i]-slope_MMstd_satperiod[i]), 
                     alpha=0.2, color='slateblue')

for mod in preprocessed_models:
    plt.scatter(np.arange(0,13,1),slopes_plot_all_satperiod[mod][1,:],label=mod, color='slateblue',marker='_', s=700)

plt.scatter(np.arange(0,13,1),slope_MMmean_satperiod,label='multi-model mean', color='k',linewidth=5,marker='_', s=400)
plt.scatter(np.arange(0,12,1),slopes_obs_79_14,label='Satellite', color='crimson',linewidth=5,marker='_', s=300)
plt.scatter(12,slope_an, color='crimson',linewidth=5,marker='_', s=300)

plt.axvline(11.5, color='k',linestyle='--')
plt.xlim(-0.75,12.75)
plt.ylim(-5.5,0.25)

plt.xticks(np.arange(0, 13, 1),month_names,fontsize=14,rotation='vertical')
plt.legend(bbox_to_anchor=[1.01, 1.01])
plt.ylabel('Slope (10$^6$ km$^2$ per $^{o}$C)', fontsize=16)
plt.title('Slope: Historical (1979-2014)', fontsize=18)

#    plt.yticks([0.5,1.5],['R$^2$','Slope'],fontsize=14)

In [ ]:
fig = plt.figure(figsize=(10,6))

r2_MMmean_satperiod = np.mean(r2_allmods_satperiod,axis=1)
r2_MMstd_satperiod = np.std(r2_allmods_satperiod,axis=1)

for i in np.arange(0,13,1):
    x = np.arange(i-0.5,i+0.59,0.1)
    y = np.ones(x.shape)
    plt.fill_between(x, y1=y*(r2_MMmean_satperiod[i]+r2_MMstd_satperiod[i]), 
                     y2=y*(r2_MMmean_satperiod[i]-r2_MMstd_satperiod[i]), 
                     alpha=0.2, color='dodgerblue')

for mod in preprocessed_models:
    plt.scatter(np.arange(0,13,1),slopes_plot_all_satperiod[mod][0,:],label=mod, color='dodgerblue',marker='_', s=700)

plt.scatter(np.arange(0,13,1),r2_MMmean_satperiod,label='multi-model mean', color='k',linewidth=5,marker='_', s=400)
plt.scatter(np.arange(0,12,1),np.power(r_obs_79_14,2),label='Satellite', color='crimson',linewidth=5,marker='_', s=300)
plt.scatter(12,r_value_annual**2, color='crimson',linewidth=5,marker='_', s=300)

plt.axvline(11.5, color='k',linestyle='--')
plt.xlim(-0.75,12.75)
plt.ylim(-0.05,1)

plt.xticks(np.arange(0, 13, 1),month_names,fontsize=14,rotation='vertical')
plt.legend(bbox_to_anchor=[1.01, 1.01])
plt.ylabel('R$^2$ value', fontsize=16)
plt.title('R$^2$ value: Historical (1979-2014)', fontsize=18)

#    plt.yticks([0.5,1.5],['R$^2$','Slope'],fontsize=14)

# Regress total SIV and Arctic mean temperature: 

In [ ]:
slopes_all_vol = {}
r_all_vol = {}
intercept_all_vol = {}

for mod in preprocessed_models:
    print('Working on '+mod+'...')
    filename = mod+'_historical_siconc_sithick_tas_pan_arctic_fields.nc'
    
    data = xr.load_dataset(os.path.join(path,filename))
    
    if len(data.sivol_arc_mean.shape)>1: 
        sivol = data.sivol_arc_mean.sel(member_id='r1i1p1f1')
    else: 
        sivol = data.sivol_arc_mean

    [slopes_all_vol[mod], 
    r_all_vol[mod],
    intercept_all_vol[mod]] = autils.scatter_linreg(data.tas_arc_mean_anom,sivol,data,all_months,mod,plotflag=False)

In [ ]:
slopes_plot_all_vol = {}
slopes_allmods_vol = np.zeros((12,len(preprocessed_models)))
r2_allmods_vol = np.zeros((12,len(preprocessed_models)))

for i,mod in enumerate(preprocessed_models):
    # Plot slopes for all models
    slopes_plot_vol = np.zeros((2,12))

    for im, m in enumerate(all_months):
        slopes_plot_vol[1,im] = slopes_all[mod][im]
        slopes_plot_vol[0,im] = r_all[mod][im]**2
            
    slopes_plot_all_vol[mod] = slopes_plot_vol
    slopes_allmods_vol[:,i] = slopes_plot_vol[1,:]
    r2_allmods_vol[:,i] = slopes_plot_vol[0,:]

In [ ]:
fig = plt.figure(figsize=(10,6))

slope_MMmean_vol = np.mean(slopes_allmods_vol,axis=1)

for mod in preprocessed_models:
    plt.scatter(np.arange(0,12,1),slopes_plot_all_vol[mod][1,:],label=mod,color='coral', marker='_', s=400)

plt.scatter(np.arange(0,12,1),slope_MMmean_vol,label='multi-model mean', color='k', marker='_', s=400, linewidth=5)
#plt.fill_between(np.arange(0,12,1), 0.2*1e11, -1.5*1e11, where=r2_MMmean_vol>r2_thresh,facecolor='royalblue', alpha=0.2)

plt.ylim(-1.5*1e11,0.2*1e11)
plt.xticks(np.arange(0, len(all_months), 1),month_names,fontsize=14,rotation='vertical')
plt.legend(bbox_to_anchor=[1.01, 1.01])
plt.ylabel('Slope (m$^3$ per $^{o}$C)', fontsize=16)
plt.title('Slope: Artic mean volume vs. surface temperature', fontsize=18)

#    plt.yticks([0.5,1.5],['R$^2$','Slope'],fontsize=14)

In [ ]:
fig = plt.figure(figsize=(10,6))

r2_thresh = 0.5
r2_MMmean_vol = np.mean(r2_allmods,axis=1)

for mod in preprocessed_models:
    plt.scatter(np.arange(0,12,1),slopes_plot_all_vol[mod][0,:],label=mod, color='royalblue',marker='_', s=400)

plt.scatter(np.arange(0,13,1),r2_MMmean_vol,label='multi-model mean', color='k',marker='_', s=400,linewidth=5)
#plt.fill_between(np.arange(0,12,1), -0.1, 1, where=slope_MMmean_vol<-0.2*1e11,facecolor='coral', alpha=0.3)

#plt.axhline(0.5, color='k',linestyle='--')

plt.ylim(-0.1, 1)
plt.xticks(np.arange(0, len(all_months)+1, 1),month_names,fontsize=14,rotation='vertical')
plt.legend(bbox_to_anchor=[1.01, 1.01])
plt.ylabel('R$^2$ value', fontsize=16)
plt.title('R$^2$ value: Artic mean volume vs. surface temperature', fontsize=18)

#    plt.yticks([0.5,1.5],['R$^2$','Slope'],fontsize=14)